<a href="https://colab.research.google.com/github/joburke1/arl_chatbot_demo/blob/main/Arlington_Missing_Middle_Demo_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#|default_exp app

Copied and modified from Anthropic cookbooks.  To see original: https://github.com/anthropics/anthropic-cookbook.git.  This is a demo of how Claude could be used to create a chat bot to provide Q&A on some of Arlington County's more complex policy reports using one of the Missing Middle research bulletins located here: https://www.arlingtonva.us/Government/Programs/Housing/Housing-Arlington/Tools/Missing-Middle/Research-Compendium

### Ingestion and calling the Claude API
The best way to pass Claude charts and graphs is to take advantage of its vision capabilities. That is, give Claude an image of the chart or graph, along with a text question about it. While all versions of Claude can accept images, Sonnet and Opus are our recommended models for data-heavy image tasks. <s> Let's get started using Sonnet.</s>  For this demo to reduce costs, we'll use haiku.

In [7]:
# Install and read in required packages, plus create an anthropic client.
%pip install anthropic IPython pdf2image

In [ ]:
# updated model to haiku to reduce costs of demo.  Added API key from colab secrets.
import base64
from anthropic import Anthropic
from IPython.display import Image
from google.colab import userdata
import os
os.environ["ANTHROPIC_API_KEY"] = userdata.get('arl-demo-key')

client = Anthropic(
)
MODEL_NAME = "claude-3-haiku-20240307"

In [ ]:
# Make a useful helper function.
def get_completion(messages):
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2048,
        temperature=0,
        messages=messages
    )
    return response.content[0].text

The best way to get a typical grapical pdf into claude is to convert each pdf page to an image. Here's how you can accomplish this.  First, add the pdf to the files for your notebook.  The original pdf can be found here: https://arlingtonva.s3.amazonaws.com/wp-content/uploads/sites/15/2020/07/MMHS_ResearchCompendium_Bulletin2_FINAL.pdf.

In [ ]:
%pip install PyMuPDF

In [ ]:
from PIL import Image
import io
import fitz

# Define the function to convert a pdf to a list of images. Note that we need to ensure we resize images to keep them within Claude's size limits.
def pdf_to_base64_pngs(pdf_path, quality=75, max_size=(1024, 1024)):
    # Open the PDF file
    doc = fitz.open(pdf_path)

    # Iterate through each page of the PDF
    for page_num in range(doc.page_count):
        # Load the page
        page = doc.load_page(page_num)

        # Render the page as a PNG image
        pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))

        # Save the PNG image
        output_path = f"../content/page_{page_num+1}.png"
        pix.save(output_path)

    # Convert the PNG images to base64 encoded strings
    images = [Image.open(f"../content/page_{page_num+1}.png") for page_num in range(doc.page_count)]
    # Close the PDF document
    doc.close()

    base64_encoded_pngs = []

    for image in images:
        # Resize the image if it exceeds the maximum size
        if image.size[0] > max_size[0] or image.size[1] > max_size[1]:
            image.thumbnail(max_size, Image.Resampling.LANCZOS)
        image_data = io.BytesIO()
        image.save(image_data, format='PNG', optimize=True, quality=quality)
        image_data.seek(0)
        base64_encoded = base64.b64encode(image_data.getvalue()).decode('utf-8')
        base64_encoded_pngs.append(base64_encoded)

    return base64_encoded_pngs

# Call the function on our Missing Middle Research Compendium
pdf_path = '../content/MMHS_ResearchCompendium_Bulletin2_FINAL.pdf' # This is the path to our slide deck stored locally.
encoded_pngs = pdf_to_base64_pngs(pdf_path)

In [ ]:
# Now let's pass the first 20 of these images (in order) to Claude at once and ask it a question about the deck. Why 20? Currently, the Anthropic API only allows you to pass in a maximum of 20 images. While this number will likely increase over time, we have some helpful tips for how to manage it later in this recipe.

content = [{"type": "image", "source": {"type": "base64", "media_type": "image/png", "data": encoded_png}} for encoded_png in encoded_pngs[:20]]
questions = "How many more housing units does Arlington need?"
content.append({"type": "text", "text": questions})
messages = [
    {
        "role": 'user',
        "content": content
    }
]

print(get_completion(messages))

According to the information provided, the Metropolitan Washington Council of Governments (COG) issued a report that highlighted the underproduction of housing in the Washington, D.C. region and the need for all jurisdictions to take actions to increase housing production. The COG Board adopted a target to add at least 320,000 housing units in the region between 2020 and 2030, with 75% of these units in Activity Centers (e.g., urban centers, growth areas, transit hubs, etc.) or near high-capacity transit, and at least 75% affordable to low- and middle-income households.
